In [1]:
%matplotlib inline

# Kaggle 대회 코드 모방하며 공부하기

## 파일 확인

In [4]:
import os

# os.walk 함수
# return value : path, path/folders, path/files
# dirname은 경로, dirname 아래 있는 폴더들, dirname 아래 있는 파일들
for dirname, folders, files in os.walk('./kaggle/dogs-vs-cats_800MB'):
    for file in files:
        print(os.path.join(dirname, file))

./kaggle/dogs-vs-cats_800MB\sampleSubmission.csv
./kaggle/dogs-vs-cats_800MB\test1.zip
./kaggle/dogs-vs-cats_800MB\train.zip


## 압축해제

In [9]:
from zipfile import ZipFile

# extractall(path) : default는 현재 작업 directory에 압축해제
# path = 압축해제할 경로
ZipFile('./kaggle/dogs-vs-cats_800MB/train.zip', 'r').extractall('./kaggle/dogs-vs-cats_800MB/')
ZipFile('./kaggle/dogs-vs-cats_800MB/test1.zip', 'r').extractall('./kaggle/dogs-vs-cats_800MB/')

## 경로 설정

In [11]:
train_path = './kaggle/dogs-vs-cats_800MB/train'
test_path = './kaggle/dogs-vs-cats_800MB/test'

## 이미지 가져오기

In [15]:
filenames = os.listdir(train_path)
categories = []

for filename in filenames:
    animaltype = filename.split('.')[0]
    categories.append(animaltype)

## DataFrame 만들기

In [19]:
import pandas as pd

dic = {'FileName':filenames, 'AnimalType':categories}
df = pd.DataFrame(dic)
df.sample(5)

,FileName,AnimalType
16404,dog.2261.jpg,dog
24210,dog.9288.jpg,dog
2102,cat.1189.jpg,cat
1988,cat.11787.jpg,cat
6376,cat.4487.jpg,cat


## 학습&검증 데이터 나누기

In [21]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)

print(train_df.shape, val_df.shape)

(20000, 2) (5000, 2)


## 데이터 증식 발생기

In [23]:
import tensorflow as tf

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator( rescale=1./255 )

In [26]:
aug_train = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_path,
    x_col='FileName',
    y_col='AnimalType',
    target_size=(224, 224),
    class_mode='binary',
    batch_size=64
)

aug_val = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=train_path,
    x_col='FileName',
    y_col='AnimalType',
    target_size=(224, 224),
    class_mode='binary',
    batch_size=64
)

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.


## Callbacks 정의

In [31]:
save_path = './SaveModel/Kaggle_Dogs&Cats.h5'

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    save_path,
    monitor='val_accuracy',
    mode='max',
    patience=9,
    verbose=1,
    save_best_only=True
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=9,
    verbose=1,
    mode='max'
)

rdlr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.1, # new_lr = lr * factor
    patience=4,
    verbose=1,
    min_lr=0.00001
)

callbacks = [checkpoint, es, rdlr]

## Model 정의

In [32]:
# kernel_initializer : 가중치 초기화 방식 설정 속성
# 종류
# LeCun 초기화 (lecun_uniform, lecun_normal) : 98년도에 얀 르쿤이 제기한 방법으로 최근에는 Xavier나 He 초기화 방식에 비해 덜 사용되는 편
# Xavier 초기화 (glorot_uniform, glorot_normal) : 케라스에서는 glorot이라는 이름으로 되어있는데, 일반적으로는 Xavier Initialization이라고 알려짐, 논문의 1저자가 Xavier Glorot임
# He 초기화 (he_uniform, he_normal) :  Kaiming He가 2015년에 제안한 가장 최신의 초기화 방식으로 수식을 보면 Xavier Initialization을 조금 개선한 것인데, 경험적으로 더 좋은 결과를 내었다고 함

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(224, 224, 3))
])